In [1]:
import json
from pybtex.database.input import bibtex

## Worker Functions

In [2]:
#Presentation function
def presentation(block):
    string = ""
    for i,author in enumerate(block["authors"]):
        if (i+1) == block["speaker"]:
            string += "{\it "+author+"}"
        else:
            string += author
        if (i+1) == len(block["authors"]):
            string += ". "
        elif (i+1) == len(block["authors"])-1:
            string += ", and "
        else:
            string += ", "
    
    string += block["title"]+". "
    string += str(block["year"])+" "+block["conference"]+', '
    string += block["address"]+". "
    if "invited" in block and block["invited"]:
        string += "{\\it Invited}."
    if "note" in block:
        string += "{\\it "+block["note"]+"}."
    
    return string

def presentations_section(pres_data):
    output = ""
    presentations = json.load(open(pres_data,mode="r"))
    for entry in presentations:
        output += presentation(entry)
        output += "\n\n"
        #output

    return output
        
#print(presentations_section("presentations.json"))

In [3]:
def publication(bibtex_data,id):
    #block = bibdata.entries[bib_id].fields
    block = bibtex_data.entries[id].fields
    authors = bibtex_data.entries[id].persons["author"]
    string = ""
    num_authors = len(authors)
    for i, author in enumerate(authors):
        #string += author.last_name[0]+", "
        #string += author.first_names[0]
        string += str(author)
        if (i == num_authors-2):
            string += " and "
        else:
            string += ", "
    string += block["title"]+". "
    string += block["year"]+". "
    string += "{\it "+block["journal"]+"}, "
    string += "{\\bf "+block["volume"]+"}, "
    page = block["pages"].split('--')[0]
    string += page+"."
    #try:
    #    string += " "+"\\url{https://doi.org/"+block["doi"]+"}"
    #except:
    #    pass
    return string

def publications_section(pubs_data):
    output = ""
    #open a bibtex file
    parser = bibtex.Parser()
    bibdata = parser.parse_file(pubs_data)

    #create a pointer array that re-orders the BibTeX entries into publication order
    pubs_order = ['' for x in range(len(bibdata.entries))]
    for (i,bib_id) in enumerate(bibdata.entries):
        b = bibdata.entries[bib_id].fields
        pubs_order[int(b["order"])-1] = bib_id
    
    #print(pubs_order)

    #loop through the individual references
    for bib_id in pubs_order:
        #b = bibdata.entries[bib_id].fields
        #print(b)
        #print(publication(bibdata,bib_id))
        #print("\n")
        output += publication(bibdata,bib_id)
        output += "\n\n"
    
    return output

#print(publications_section("publications.bib"))

## Test Section

In [10]:
#Multiple Blocks via the "Sequence" entry
design_data = json.load(open("design.json",mode='r'))

header = open("header.tex",mode='r')
output = open("newcv.tex",mode='w')
for line in header:
    output.write(line)
header.close()

for section in design_data["Sequence"]:
    #Extract the current section
    data_block = design_data[section]
    if isinstance(data_block["number"],int):
        how_many = data_block["number"]
    else:
        how_many = 1e6 #I don't think I'll have more than 1 million of any category
    #Write the current section
    output.write("\section{\sc "+data_block["label"]+'}\n')
    if data_block["label"] == "Publications":
        output.write(publications_section(data_block["file"]))
    elif data_block["label"] == "Conference Presentations":
        output.write(presentations_section(data_block["file"]))
    elif data_block["label"] == "Invited Seminars":
        output.write("WRITE THIS MODULE")
    else:
        if how_many < 1e6:
            output.write(str(how_many)+" Most Recent "+data_block["label"]+"\n\n")
        input_data = open(data_block["file"])
        items = []
        for line in input_data:
            if line != '\n':
                items.append(line.strip('\n'))
        input_data.close()
        for (count,item) in enumerate(items):
            if count < how_many:
                #print item
                output.write(item+'\n'+'\n')
    

#Finalize the TeX document
output.write("\end{resume}\n\end{document})\n")
output.close()

In [ ]:
TO-DO:
    1. Option for a TeX block, not list
    2. Education section
    3. Research Interests
    4. Journal reviewing
    5. Conference organization
    6. Service activities
    7. Contact Information

Tasks:
    1. Convert presentation list to JSON. DONE
    2. Convert publications list to BibTeX. DONE
    3. Identify BibTeX interpretor and write function for TeX output. DONE
    4. Write function for seminars [crib from presentation?]
    
    